# path_helpers

> Converting SVG paths to a series of points

In [ ]:
#| default_exp stroke3.path_helpers

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np


def to_points(parsed_path, n):
    prev = None
    acc = []
    step_size = 1.0 / n
    for x in np.arange(0.0, 1.0, step_size):
        _complex_pt = parsed_path.point(x)
        _x = _complex_pt.real
        _y = _complex_pt.imag
        pt = (_x, _y)
        acc.append(pt)
    return acc

In [ ]:
#| export
import numpy as np


def discretize_path(path, total_n=1000, min_n=3):
    chunk_size = path.length() / total_n
    subpaths = []
    for subpath in path.continuous_subpaths():
        sub_n = subpath.length() / chunk_size
        # drop paths shorter than the minimum number of points.
        if sub_n < min_n:
            continue
        # TODO: consider joining subpaths if the next start point is within 'dist' of this endpoint
        subpaths.append(np.array(to_points(subpath, n=sub_n)))
    return np.array(subpaths, dtype=object)

In [ ]:
#| export
def discretize_paths(paths, total_n=1000, min_n=3):
    all_strokes = []
    total_length = sum([p.length() for p in paths])
    for p in paths:
        # compute number of points to generate for curr path based on ratio to total path length
        path_n = int(total_n * p.length() / total_length)
        # if number of points is less than the minimum, drop it.
        if path_n < min_n:
            # print(f"skipping path - path_n={path_n}, path_length={p.length()}, total_length={total_length}")
            continue
        # print(f"discretize path - path_n={path_n}, path_length={p.length()}, total_length={total_length}")

        strokes = discretize_path(p, path_n, min_n=min_n)
        all_strokes.extend(strokes)
    return all_strokes

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()